# LAB2
### Жерлыгин Максим
### М8О-408Б-18

Скачать датасет с Kaggle (https://www.kaggle.com/competitions/tensorflow-great-barrier-reef/data). <br>
Подключить гугл диск к колабу.<br>
Закинуть на гугл диск часть изображений из скачанного датасета (train_images/video_0).<br>
Сгенерировать лейблы из файла train.csv такого формата, который нужен для обучения YOLO (только для video 0). Для каждого изображения из video_0 должен быть txt файл с таким же именем, который будет содержать разметку для всех объектов этого изображения<br>
Сложить данные так, как нужно для обучения модели (таким образом, чтобы при замене images в пути к картинкам на lables получались пути к лейблам, см. выше)<br>
Внести изменения в пути и количество классов и их название в файле coco128.yaml (или создать новый файл на подобие coco128.yaml, в котором будут пути к данным и тд) <br>
Запустить обучение YOLOv5 на наших данных. 



In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install
import os 
import pandas as pd 
import random
import shutil
import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-176-gaa7a0e9 torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.2/78.2 GB disk)


In [2]:
# Подключаемя к гугл диску
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/lab_3/train.csv')
pics = os.listdir('/content/drive/MyDrive/lab_3/'+f'video_0/')
i_a = {int(dataset.iloc[i]['image_id'][2:]): eval(dataset.iloc[i]['annotations']) for i in list(dataset.index) if eval(dataset.iloc[i]['annotations'])!=[] and dataset.iloc[i]['image_id'][2:]+'.jpg' in pics}


In [ ]:
val = random.sample(list(i_a.keys()), int(len(list(i_a.keys()))) * 0.2)
train = list(set(list(i_a.keys())) - set(val))

In [ ]:
os.mkdir('/content/datasets/')
os.mkdir('/content/datasets/custom_dataset/')
os.mkdir('/content/datasets/custom_dataset/images')
os.mkdir('/content/datasets/custom_dataset/images/train')
os.mkdir('/content/datasets/custom_dataset/images/val')
os.mkdir('/content/datasets/custom_dataset/labels')
os.mkdir('/content/datasets/custom_dataset/labels/train')
os.mkdir('/content/datasets/custom_dataset/labels/val')

In [ ]:
for i in val:
  shutil.copy('/content/drive/MyDrive/lab_3/'+f'video_0/{i}.jpg','/content/datasets/custom_dataset/images/val')
  with open(f"/content/datasets/custom_dataset/labels/val/{i}.txt", "w+") as f:
    for k in range(len(i_a[i])-1):
      f.write(f"0 {i_a[i][k]['x']/1280} {i_a[i][k]['y']/720} {i_a[i][k]['width']/1280} {i_a[i][k]['height']/720}\n")
    f.write(f"0 {i_a[i][-1]['x']/1280} {i_a[i][-1]['y']/720} {i_a[i][-1]['width']/1280} {i_a[i][-1]['height']/720}")

In [ ]:
for i in train:
  shutil.copy('/content/drive/MyDrive/lab_3/'+f'video_0/{i}.jpg','/content/datasets/custom_dataset/images/train')
  with open(f"/content/datasets/custom_dataset/labels/train/{i}.txt", "w+") as f:
    for k in range(len(i_a[i])-1):
      f.write(f"0 {i_a[i][k]['x']/1280} {i_a[i][k]['y']/720} {i_a[i][k]['width']/1280} {i_a[i][k]['height']/720}\n")
    f.write(f"0 {i_a[i][-1]['x']/1280} {i_a[i][-1]['y']/720} {i_a[i][-1]['width']/1280} {i_a[i][-1]['height']/720}")

In [ ]:
!python train.py --img 1280 --batch 16 --epochs 30 --data /content/datasets/custom_dataset/my_file.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=/content/datasets/custom_dataset/my_file.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-164-g177da7f torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.